# OpenAI API

Note: `openai.ChatCompletion` needs the latest version of the API (v0.27.0+), update with `pip install --upgrade openai`

In [ ]:
import openai
import os
import pandas as pd
import regex
import sys
import time
sys.path.append(os.getcwd() + '/..')
from scripts import read_transkribus_files

In [ ]:
def get_openai_key(keyfile):
    infile = open(keyfile, "r")
    lines = infile.readlines()
    infile.close()
    return lines[-1].split("=")[-1].strip()

In [ ]:
KEYFILE = "../../openai-quickstart-python/.env"

In [ ]:
openai.api_key = get_openai_key(KEYFILE)

In [ ]:
#data_dir = "../../data/Overlijden/x-samples/three-columns-100/page"

#texts, metadata, textregions = read_transkribus_files.read_files(data_dir)

In [ ]:
csv_file_df = pd.read_csv("three_columns_100_corrected.csv")
records = csv_file_df.to_dict(orient="records")
texts = { record["Unnamed: 0"]: record["0"] for record in records }

In [ ]:
PROMPT_DECEASED_NAME = """Here is a Dutch death certificate. 
Can you extract the name of the deceased person from this document? 
Only give the deceased name as result. 
Other information from the document is not necessary.
""" 

PROMPT_DECEASED_FIRST_NAME = """Here is a Dutch death certificate. 
Can you extract the first name of the deceased person from this document? 
Only give the deceased first name as result.
Other information from the document is not necessary.
"""

PROMPT_DECEASED_LAST_NAME = """Here is a Dutch death certificate. 
Can you extract the last name of the deceased person from this document? 
Only give the deceased last name as result.
Other information from the document is not necessary.
""" 

PROMPT_DEATH_DATE = """Here is a Dutch death certificate. 
Can you extract the death date from this document? 
Please present the result in numeric form, with a zero prepending day and month numbers smaller than 10, so for example like: 01-01-1900.
Only give the death date as result. 
Other information from the document is not necessary.
""" 

PROMPT_MOTHER_NAME = """Here is a Dutch death certificate. 
Can you extract the name of the mother of the deceased person from this document? 
Only give the mother's name as result. 
Other information from the document is not necessary.
""" 

In [ ]:
START_KEY = ""
PROMPT = PROMPT_MOTHER_NAME

def process_key(text, prompt, key):
    response = openai.ChatCompletion.create(
       model="gpt-3.5-turbo",
       messages=[
          {"role": "user", "content": prompt + text },
       ]
    )
    responses[key] = response.choices
    print(responses[key][0]["message"]["content"], key)
    time.sleep(1)
    

if START_KEY == "":
    responses = {} 
for key in sorted(texts.keys()):
    if key >= START_KEY:
        try:
            process_key(texts[key], PROMPT, key)
        except:
            time.sleep(5)
            process_key(texts[key], PROMPT, key)

In [ ]:
for text_id in responses:
    #if regex.search(" not ", responses[text_id][0]["message"]["content"]):
    print(text_id, responses[text_id][0]["message"]["content"], sep=",")

In [ ]:
for key in sorted(texts.keys()):
    first_name = responses[key+"_first"][0]["message"]["content"]
    first_name = re.sub('[".]', "", first_name)
    print(first_name, end = "")
    if not re.search(" not ", responses[key+"_last"][0]["message"]["content"]):
        last_name = responses[key+"_last"][0]["message"]["content"]
        last_name = re.sub("The deceased last name is ", "", last_name)
        last_name = re.sub(" \(no last name given\)", "", last_name)
        last_name = re.sub('[".]', "", last_name)
        last_name = re.sub('-$', "", last_name)
        first_name_parts = first_name.split()
        if last_name != first_name_parts[-1]:
            print(f" {last_name}", end="")
    print()